In [1]:
# General
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns


# EDA
from pandas_profiling import ProfileReport

# With missforest we can use random forest to impute data. This is better than mean by far
# to avoid errors on loading missingpy
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

# Outliers with isolation forest.
from sklearn.ensemble import IsolationForest

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics

# Model to apply
import lightgbm as lgb


Bad key figure.labelsize in file /Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/classic 2.mplstyle, line 310 ('figure.labelsize:   medium    # size of the figure label')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key figure.labelweight in file /Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/classic 2.mplstyle, line 311 ('figure.labelweight: normal    # weight of the figure label')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution


## Loading data

In [2]:
# loading data
train = pd.read_csv('../csv_data/train.csv', sep = ',')
test = pd.read_csv('../csv_data/test.csv', sep = ',')

## Loading test and training data together

This is optional. 
- This would be to make all the transformations at the same time instead of dealing with training and test sets in a different way. 
- You don't need to execute this code if there is no train - test datasets, cause you'll have only one dataset.
- This can be done serarately and can return different results.

In [3]:
SalePrice = train.SalePrice
data_frame = pd.concat([train.drop('SalePrice', axis=1), test], ignore_index = True)

data_frame
SalePrice

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

## Creating pandas profiling on folder reports

In [60]:
# pandas profiling
profile = ProfileReport(data_frame, title = 'Pandas Profiling Report')
# profile to widgets
# profile.to_widgets()

# profile to html
# profile.to_notebook_iframe()

# exporting profile to a external file. Best option
profile.to_file('../reports/train_test/your_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/pandas_profiling/model/correlations.py:55: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/missingno/missingno.py:254: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculati

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Fixing columns with 0s that are not correctly marked as missing values

In [4]:
# select manually the columns with numerical or categorical data to change 0 with NaN
zeros_col_list = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

# replace categorial or numerical 0s with Nan
data_frame[zeros_col_list] = data_frame[zeros_col_list].replace({'0' : np.nan, 0 : np.nan})

## New profile with no 0s

In [62]:
# pandas profiling
profile_no_zeros = ProfileReport(data_frame, title = 'Pandas Profiling Report')

# exporting profile to a external file. Best option
profile_no_zeros.to_file('../reports/train_test/your_report_no_zeros.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/pandas_profiling/model/correlations.py:55: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/missingno/missingno.py:254: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculati

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Removing columns with a give ratio of NaN values

In [5]:
# if we set a threshold of 0.3 we are removing columns with Nan ratio of 70%
data_frame_clean = data_frame.dropna(axis=1, thresh = int(0.3 * data_frame.shape[0]))

# New Pandas profiling with colums removed

In [64]:
# pandas profiling
profile_removed_na_columns = ProfileReport(data_frame_clean, title = 'Pandas Profiling Report')

# exporting profile to a external file. Best option
profile_removed_na_columns.to_file('../reports/train_test/your_report_removed_na_columns.html')

/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/multimethod/__init__.py:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/missingno/missingno.py:254: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Count": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'The number of FixedLocator locations (7), usually from a call to set_ticks, does not match the number of ticklabels (69).')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Transforming data at once.

In this example we need to make this process in the same script since the data will be transformed all together (train and test). Afterwards, those would be separated.

## Label encoder to categorical columns

This is done so we can apply algorithms either to impute or to predict to our dataset

In [6]:
# taking a look to categorical variables
cat_vars = data_frame_clean.select_dtypes(exclude=[np.number])
cat_vars.describe()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
count,2915,2919,2919,2919,2917,2919,2919,2919,2919,2919,...,2918,2917,1499,2762,2760,2760,2760,2919,2918,2919
unique,5,2,4,4,2,5,3,25,9,8,...,4,7,5,6,3,5,5,3,9,6
top,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal
freq,2265,2907,1859,2622,2916,2133,2778,443,2511,2889,...,1492,2717,744,1723,1230,2604,2654,2641,2525,2402


In [7]:
# process columns, apply LabelEncoder to categorical features
for c in cat_vars.columns:
    lbl = LabelEncoder() 
    lbl.fit(list(data_frame_clean[c].values)) 
    data_frame_clean[c] = lbl.transform(list(data_frame_clean[c].values))



/var/folders/yj/nrb4536x26s8lfxkz_6kzf100000gn/T/ipykernel_67553/857206388.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame_clean[c] = lbl.transform(list(data_frame_clean[c].values))


## Impute categorical data with Random forest algorithm

This chunk is different from the other notebooks. 
Predicted column doesn't need to be spllited since it was done at the begining
When merging dataframes we are going to separate train from test and add SalePrice to train dataset.

In this way we'll have all the data transformed and ready to train. Any kind of transformation has to be done before this step.

In [8]:
# Make an instance and perform the imputation
imputer = MissForest(max_iter = 10, max_features=1.0, criterion='squared_error')
data_frame_imputed = pd.DataFrame(imputer.fit_transform(data_frame_clean))

# Renaming columns after imputation
data_frame_imputed.columns = data_frame_clean.columns

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6


## Remove outliers with Isolation Forest (Random forest algorithm)

This has to be done before imputing data because the algorithm doesn't accept nan

In [76]:
data_frame_imputed_no_outliers = data_frame_imputed.copy()

# to get all the columns except the Id column
for column in data_frame_imputed_no_outliers.drop('Id', axis = 1).columns:
    
    # initiating the model and fitting the column selected
    model = IsolationForest(n_estimators = 50, max_samples = 'auto', contamination = float(0.1), max_features = 1.0)
    model.fit(data_frame_imputed_no_outliers[[column]])

    # gonna save a scores and anomaly column regarding to the column selected. This would be for the manual process
    data_frame_imputed_no_outliers['scores'] = model.decision_function(data_frame_imputed_no_outliers[[column]])
    data_frame_imputed_no_outliers['anomaly'] = model.predict(data_frame_imputed_no_outliers[[column]])

    # for the given column if anomaly is equal -1 transform value to NaN
    data_frame_imputed_no_outliers[column] = np.where(data_frame_imputed_no_outliers["anomaly"] == -1 , np.nan, data_frame_imputed_no_outliers[column])
    
    # dropping scores and anomaly columns from dataframe
    data_frame_imputed_no_outliers = data_frame_imputed_no_outliers.drop(['scores', 'anomaly'], axis = 1)


/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have val

## Pandas profiling

We are going to check what changes in out data when removed the outliers.

In [77]:
# pandas profiling
profile_removed_ouliers = ProfileReport(data_frame_imputed_no_outliers, title = 'Pandas Profiling Report')

# exporting profile to a external file. Best option
profile_removed_ouliers.to_file('../reports/train_test/your_report_removed_ouliers.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/missingno/missingno.py:254: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Count": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'The number of FixedLocator locations (7), usually from a call to set_ticks, does not match the number of ticklabels (69).')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## In doing the outliers step we found out in pandas profiling that several columns remained with just one value.

We are going to remove those columns.

In [78]:
#check that this is similar to pandas profiling list
constant_cols = data_frame_imputed_no_outliers.columns[data_frame_imputed_no_outliers.nunique() <= 1]

data_frame_imputed_no_outliers = data_frame_imputed_no_outliers.drop(constant_cols, axis = 1)


## Pandas profiling to check that all constant columns are removed

In [79]:
# pandas profiling
profile_removed_ouliers_no_constant = ProfileReport(data_frame_imputed_no_outliers, title = 'Pandas Profiling Report')

# exporting profile to a external file. Best option
profile_removed_ouliers_no_constant.to_file('../reports/train_test/your_report_removed_ouliers_no_constant.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/missingno/missingno.py:254: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Count": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'The number of FixedLocator locations (7), usually from a call to set_ticks, does not match the number of ticklabels (56).')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Reimputation of data.

Since we've erased some outliers, now we are going to impute them.

In [80]:
# Make an instance and perform the imputation
imputer = MissForest(max_iter = 10, max_features=1.0, criterion='squared_error')
data_frame_imputed_2 = pd.DataFrame(imputer.fit_transform(data_frame_imputed_no_outliers))

# Renaming columns after imputation
data_frame_imputed_2.columns = data_frame_imputed_no_outliers.columns


Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,1.0,60.0,3.0,65.0000,8450.00,3.0,3.0,4.00,5.00,2.0,...,1.00,2.0000,548.00,4.0,207.4000,61.00,2.00,2008.0,8.0,4.0
1,2.0,20.0,3.0,80.0000,9600.00,3.0,3.0,2.76,14.31,1.0,...,1.00,2.0000,460.00,4.0,298.0000,109.67,5.00,2007.0,8.0,4.0
2,3.0,60.0,3.0,68.0000,11250.00,0.0,3.0,4.00,5.00,2.0,...,1.00,2.0000,608.00,4.0,191.4600,42.00,9.00,2008.0,8.0,4.0
3,4.0,70.0,3.0,60.0000,9550.00,0.0,3.0,0.00,6.00,2.0,...,2.00,3.0000,642.00,4.0,233.3800,35.00,2.00,2006.0,8.0,4.0
4,5.0,60.0,3.0,84.0000,14260.00,0.0,3.0,3.68,15.00,2.0,...,1.00,3.0000,836.00,4.0,192.0000,84.00,7.47,2008.0,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915.0,119.7,4.0,58.9553,8857.03,3.0,3.0,4.00,13.06,2.0,...,1.22,2.3100,651.02,5.0,210.0000,58.65,6.00,2006.0,8.0,4.0
2915,2916.0,120.0,4.0,40.9659,3993.14,3.0,3.0,4.00,12.53,2.0,...,2.00,1.0000,286.00,4.0,218.6400,24.00,4.00,2006.0,8.0,4.0
2916,2917.0,20.0,3.0,69.2063,9189.81,3.0,3.0,4.00,11.00,2.0,...,2.00,2.0000,576.00,4.0,238.4418,145.30,9.00,2006.0,8.0,4.0
2917,2918.0,20.0,3.0,62.0000,10441.00,3.0,3.0,4.00,11.00,2.0,...,1.22,2.0223,574.38,5.0,80.0000,32.00,7.00,2006.0,8.0,4.0


## Pandas profiling to check that the imputation was done correctly

In [83]:
# pandas profiling
profile_imputed_2 = ProfileReport(data_frame_imputed_2, title = 'Pandas Profiling Report')

# exporting profile to a external file. Best option
profile_imputed_2.to_file('../reports/train_test/your_report_imputed_2.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/missingno/missingno.py:254: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(
/Users/hectorcozar/Documents/GitHub/templates/venv/lib/python3.9/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Count": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'The number of FixedLocator locations (7), usually from a call to set_ticks, does not match the number of ticklabels (56).')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Split data again on train-test datasets

Separating datasets and adding back the predicted variable to the train set.

In [9]:
# get train and test final dataframes transformed ready to get into the model.
train_final = pd.merge(data_frame_imputed_2[0:len(SalePrice)], SalePrice, how = 'left', right_index = True, left_index = True)
test_final = data_frame_imputed_2[len(SalePrice):].reset_index(drop = True)


## Storing cleaned dataframe

In [10]:
%store train_final
%store test_final

Stored 'train_final' (DataFrame)
Stored 'test_final' (DataFrame)
